# Import necessary packages

In [9]:
from tradingview_ta import TA_Handler, Interval, Exchange
import numpy as np
import schedule
from schedule import run_pending
import time
import datetime
import os
import json
import requests

# Time Zone Setup

In [8]:
#----------------------------Time Setup UTC--------------------------------#
# os.environ["TZ"] = "UTC"
# time.tzset()

# Telegram Bot and Channel Token and username

In [3]:
#--------------------------Telegram Bot Setup------------------------------#
from telegram import Bot
bot = Bot('YOUR TELEGRAM BOT TOKEN ')
channel_id_ForexCrypto="@TEKEGRAM CHANNEL USERNAME"

## Adding symbols

In [4]:
#----------------------------------------------------------------------#
#     In this section all important symbols were added from
#          Binance and FXCM which are two biggest
#            brokers in crypto and forex market
Binance_Symbols_List = ['BTCUSDT','ETHUSDT','MATICUSDT','SOLUSDT'
                        ,'ADAUSDT','AVAXUSDT','XRPUSDT','BNBUSDT'
                        ,'DOTUSDT','SHIBUSDT','AAVEUSDT','SANDUSDT'
                        ,'GMTUSDT','ATOMUSDT','FTMUSDT','DOGEUSDT'
                        ,'DYDXUSDT','NEARUSDT','ETHBTC','UNIUSDT'
                        ,'GALAUSDT','RUNEUSDT','LINKUSDT','APEUSDT'
                        ,'CRVUSDT','MANAUSDT','EGLDUSDT','TRXUSDT'
                        ,'WAVESUSDT','NKNUSDT','FIROUSDT','UNFIUSDT'
                        ,'ALGOUSDT','LTCUSDT','SRMUSDT','ICPUSDT'
                        ,'BURGERUSDT','QNTUSDT','XLMUSDT','VGXUSDT'
                        ,'JASMYUSDT','QUICKUSDT','GRTUSDT','XTZUSDT'
                        ,'LRCUSDT','SNXUSDT','ETCUSDT','COMPUSDT'
                        ,'AMPUSDT','KNCUSDT','ACHUSDT','FILUSDT'
                        ,'ETHBTC','SOLBTC','LTCBTC','LTCETH'
                        ,'SOLETH','DOGEBTC','ADABTC','XRPBTC'
                        ,'EOSUSDT','FETUSDT','KSMUSDT','CELRUSDT'
                        ,'ARUSDT','ALICEUSDT','IRISUSDT','ROSEUSDT'
                        ,'ALPHAUSDT',]

FXCM_Symbols_List    =  ['EURUSD','GBPUSD','AUDUSD','USDCAD'
                        ,'USDCHF','USDJPY','NZDUSD','EURAUD'
                        ,'EURJPY','GBPJPY','EURCHF']

In [17]:
## Set variables for CCI indicators
cci_status_previous_forex = np.zeros(len(FXCM_Symbols_List))
cci_status_previous_forex = cci_status_previous_forex.astype(int)
cci_status_previous_crypto = np.zeros(len(Binance_Symbols_List))
cci_status_previous_crypto = cci_status_previous_crypto.astype(int)

## Detecting signals using CCI oscilators

In [6]:
#----------------------------------------------------------------------#
#            This is a function that based on Oscillators
#      which detect Long and Short signals using CCI with 20 Period
#        returning -1 means Short Alert and returning +1 means Long
def signal_check(trend_status: int, cci_signal_status: int):
    '''
    This function gets Trend and Oscilators  status and returns
    Three different values:
     1  : Long  Signal
    -1  : Short Signal
     0  : No    Signal
    '''
    signal_final = 0
    if (trend_status == 1) and (cci_signal_status == 1):
        signal_final = 1  # Long Position

    elif (trend_status == -1) and (cci_signal_status == -1):
        signal_final = -1  # Short Position
    else:
        signal_final = 0
    # Reset indicators values
    cci_signal_status = 0
    trend_status = 0

    return signal_final

## Detecting Trend using Slow and Fast EMA

In [7]:
#----------------------------------------------------------------------#
#            This is a function that based on Moving averages
#             which detect Trend Using EMA(50) and EMA(20)
#        returning -1 means Downtrend and returning 1 means Uptrend
def trend_detecting(ema_200 : float, ema_50: float):
    trend_flag = 0
    if ema_200 > ema_50 :
        trend_flag = -1  # DownTrend
    elif ema_200 < ema_50:
        trend_flag = 1   # UpTrend
    else:
        pass
    # print('Trend Status : ',trend_flag)
    return trend_flag

## Detecting signals based on CCI and Trend functions output

In [11]:
#-----------------------------Trend and Oscillators signal check-------------------------------#
#           we only detecting sell signals in cci when we have a downtrend and
#                 detectign buy signals in cci when we have a uptrend
import numpy as np
def cci_signal_detecting(cci_value: float , iteration_step : int, cci_status_previous: np.ndarray):
    '''
    This function is based on Oscillators
    which detect Long and Short signals using CCI with 20 Period
    returning -1 means Short Alert and returning +1 means Long
    '''
    if   cci_value > 100 :
        cci_status = 1
    elif cci_value < -100:
        cci_status = -1
    else:
        cci_status = 0

    if (cci_status == 1 and (cci_status_previous[iteration_step] == -1 or cci_status_previous[iteration_step] == 0)):
        signal_status = 1
        cci_status_previous[iteration_step] = 1
    elif (cci_status == -1 and (cci_status_previous[iteration_step] == 1 or cci_status_previous[iteration_step] == 0)):
        signal_status = -1
        cci_status_previous[iteration_step] = -1
    else:
        signal_status = 0
    # print('cci_status:',cci_status,'cci_previous:',cci_status_previous,'signal cci :',signal_status)

    return signal_status


# Send to telegram channel

In [12]:
def send_to_telegram(signal_final: int,
                     symbol_name: str,
                     screener_name: str,
                     exchange_name: str,
                     timeframe: str,
                     open_price = None,
                     close_price = None
                     ):
    '''
    This function Sends the signal to Telegram Channel
    '''
    try:
        if (screener_name == "crypto") or (screener_name == "forex"):
            if signal_final == 1:
                message = screener_name.capitalize() + " " + "Signal" + '\U0000269C' + "\n\n" + \
                          symbol_name + \
                          "\n" + "Buy at " + str(close_price)+ '\U00002B06' + \
                          "\n\n" + "@AbayFx_ForexCrypto"
                print(message)
                telegram_out = bot.send_message(channel_id_ForexCrypto, message)
                print(telegram_out)
            elif signal_final == -1:
                message = screener_name.capitalize() + " " + "Signal" + "\U0000269C" + "\n\n" + \
                          symbol_name + \
                          "\n" + "Sell at " + str(close_price)  + "\U00002B07" + \
                          "\n\n" + "@AbayFx_ForexCrypto"
                print(message)
                telegram_out = bot.send_message(channel_id_ForexCrypto, message)
                print(telegram_out)
            else:
                pass

        else:
            pass

    except:
        print('Error Happend!')


## Market Scanner

In [13]:
#---------------------------Crypto Scanner Function--------------------------------#
def market_scanner(symbols_list     : str,
                   screen_name      : str,
                   exchange_name    : str,
                   timeframe        : str,
                   cci_previous_status  :  np.ndarray,
                   indicator_1      ='EMA200',
                   indicator_2      ='EMA50',
                   indicator_3      ='CCI20',
                   indicator_4      ='close',
                   indicator_5      ='open'):
    '''
    This function scan all symbols you gave and detect that whether it has a potential for signals or not
    :param symbols_list: The all symbols this function scan and extract their indicators
    :param screen_name: symbols have different kind of screen name like crypto and forex
    :param exchange_name: specify which exchange symbols list work with
    :param timeframe: set in which timeframe should receive indicators value
    :param indicator_1:
    :param indicator_2:
    :param indicator_3:
    :param indicator_4:
    :param indicator_5:
    :return:
    '''
    for iteration, symbol in enumerate(symbols_list):
        try:
            Outcome = TA_Handler(
                symbol=symbol,
                screener=screen_name,
                exchange=exchange_name,
                interval=timeframe
            )

            slow_ema = Outcome.get_analysis().indicators[indicator_1]
            fast_ema = Outcome.get_analysis().indicators[indicator_2]
            cci_output = Outcome.get_analysis().indicators[indicator_3]
            Indicator_4_Value = Outcome.get_analysis().indicators[indicator_4]
            Indicator_5_Value = Outcome.get_analysis().indicators[indicator_5]
            trend_status = trend_detecting(slow_ema, fast_ema)
            signal_cci = cci_signal_detecting(cci_output, iteration, cci_previous_status)
            signal_final = signal_check(trend_status=trend_status, cci_signal_status=signal_cci)
            send_to_telegram(signal_final, symbol_name= symbol, screener_name= screen_name, exchange_name= exchange_name
                             , timeframe= timeframe,close_price=Indicator_4_Value)

            print('SYMBOL:',symbol)
            print('Buy signal ,' if signal_final == 1 else ("Sell signal ," if signal_final == -1 else "No signal ,"),
                  'Uptrend ,' if trend_status == 1 else 'Downtrend ,',
                  ' signal CCI long ' if signal_cci == 1 else ' signal CCI short')

            # print(Outcome.get_analysis().indicators)


        except:
            print("Error happend in market scanner!", symbol)


# Schedule Running

In [14]:
#--------------- Run job every minute/hour at the 2rd second ---------------#
# schedule.every().minute.at(":59").do(crypto_scanner)
# schedule.every().hour.at("59:55").do(market_scanner, symbols_list = FXCM_Symbols_List,
#                                      screen_name="forex" , exchange_name="FX_IDC",
#                                      timeframe= Interval.INTERVAL_1_HOUR, cci_previous_status = cci_status_previous_forex )

# Main Function

In [15]:
#------------------If you want to run the code on server uncomment this section-----------------------#
# if __name__ == '__main__':
#     # main()
#     while True:
#         schedule.run_pending()
#         time.sleep(0.01)

In [18]:
# Test

market_scanner(symbols_list = FXCM_Symbols_List,
                                     screen_name="forex" , exchange_name="FX_IDC",
                                     timeframe= Interval.INTERVAL_1_HOUR, cci_previous_status = cci_status_previous_forex)

Forex Signal⚜

EURUSD
Sell at 1.07415⬇

@AbayFx_ForexCrypto
<coroutine object Bot.send_message at 0x7f10fefc1bd0>
SYMBOL: EURUSD
Sell signal , Downtrend ,  signal CCI short


<ipython-input-13-b8ad58c9533b>:42: RuntimeWarning: coroutine 'Bot.send_message' was never awaited
  send_to_telegram(signal_final, symbol_name= symbol, screener_name= screen_name, exchange_name= exchange_name


SYMBOL: GBPUSD
No signal , Downtrend ,  signal CCI short
SYMBOL: AUDUSD
No signal , Downtrend ,  signal CCI short
SYMBOL: USDCAD
No signal , Uptrend ,  signal CCI short
Forex Signal⚜

USDCHF
Buy at 0.88831⬆

@AbayFx_ForexCrypto
<coroutine object Bot.send_message at 0x7f10fefc2570>
SYMBOL: USDCHF
Buy signal , Uptrend ,  signal CCI long 
Forex Signal⚜

USDJPY
Buy at 147.23⬆

@AbayFx_ForexCrypto
<coroutine object Bot.send_message at 0x7f10fefc25e0>
SYMBOL: USDJPY
Buy signal , Uptrend ,  signal CCI long 
Forex Signal⚜

NZDUSD
Sell at 0.58775⬇

@AbayFx_ForexCrypto
<coroutine object Bot.send_message at 0x7f10fefc2420>
SYMBOL: NZDUSD
Sell signal , Downtrend ,  signal CCI short
SYMBOL: EURAUD
No signal , Downtrend ,  signal CCI short
SYMBOL: EURJPY
No signal , Downtrend ,  signal CCI short
SYMBOL: GBPJPY
No signal , Uptrend ,  signal CCI short
Forex Signal⚜

EURCHF
Sell at 0.95415⬇

@AbayFx_ForexCrypto
<coroutine object Bot.send_message at 0x7f10fefc25e0>
SYMBOL: EURCHF
Sell signal , Downtrend